# SQL Quick Insights for Music Store

## Purpose
The purpose of this project is to demonstrate application of SQL queries to answer current questions of the organization to guide immediate action.

## Context
[Chinook](https://github.com/lerocha/chinook-database), a hypothetical record store, has hired our analytics team to assist their decision-making.  We are provided access to the company's SQLite database and schema and assigned the following tasks:

1\. Of the following 4 artists of a newly signed record label, which 3 should have albums added to the store based on top-selling genres in the USA?  

| Artist Name | Genre |
| ----------- | ----- |
| Regal | Hip-Hop |
| Red Tone | Punk |
| Meteor and the Girls | Pop |
| Slim Jim Bites | Blues |

2\. Based on analysis of each sales agent's customer purchases, are any sales agents outperforming or underperforming the group?

3\. Provide a customer sales analysis by country, including:
- total number of customers
- total value of sales
- average value of sales per customer
- average order values

4\. Analyze indivual track vs whole album purchases to advise whether shifting to offering only individual most popular tracks would increase ROI.  The company has instructed that albums with only a few total tracks may be excluded from the analysis.

### Import Necessary Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3 as sql

### Python Functions to Run SQL Queries

#### Function to RUN SQL Query and Return Pandas DataFrame

In [2]:
def run_query(query):
    '''
    Takes input SQL query string and returns
    results from chinook.db in a pandas dataframe
    '''
    with sql.connect('chinook.db') as conn:
        return pd.read_sql(query, conn)

#### Function to Run Execute SQL without Returning Display (e.g., CREATE VIEW)

In [3]:
def run_command(command):
    '''
    Executes SQL command string on chinook.db
    '''
    with sql.connect('chinook.db') as conn:
        conn.isolation_level = None
        conn.execute(command)

#### Function to Display Current Tables and Views in SQL Database

In [4]:
def show_tables():
    '''
    Returns current pandas dataframe of all tables 
    and views in database by calling run_query(query) function
    '''
    query = 'SELECT name, type FROM sqlite_master \
             WHERE type IN ("table", "view")'
    return run_query(query)

#### Checking Initial State of Database

In [5]:
# Running show_tables() function to view initial tables of database
show_tables()

,name,type
0,album,table
1,artist,table
2,customer,table
3,employee,table
4,genre,table
5,invoice,table
6,invoice_line,table
7,media_type,table
8,playlist,table
9,playlist_track,table


#### Observations:
- Chinook's database has 11 tables.

## Selecting Albums to Purchase

#### TOP 10 Genres Sold in the USA

In [6]:
# Defining SQL query as string
query = '''
WITH invoice_line_usa AS
    (
    SELECT *
      FROM invoice_line as il
           INNER JOIN invoice as i
           ON i.invoice_id = il.invoice_id
           
           INNER JOIN customer as c
           ON c.customer_id = i.customer_id
     WHERE c.country = 'USA'
    )

SELECT g.name as genre,
       COUNT(*) AS total_tracks_usa,
       ROUND((COUNT(*) / CAST((SELECT COUNT(*) FROM invoice_line_usa) AS FLOAT)) * 100, 1) AS percentage_usa
  FROM track as t
       INNER JOIN invoice_line_usa as ilu
       ON ilu.track_id = t.track_id
       
       INNER JOIN genre as g
       ON g.genre_id = t.genre_id
 GROUP BY genre
 ORDER BY total_tracks_usa DESC
 LIMIT 10;
'''
# Querying database with run_query function
usa_top_genres = run_query(query)
usa_top_genres

,genre,total_tracks_usa,percentage_usa
0,Rock,561,53.4
1,Alternative & Punk,130,12.4
2,Metal,124,11.8
3,R&B/Soul,53,5.0
4,Blues,36,3.4
5,Alternative,35,3.3
6,Latin,22,2.1
7,Pop,22,2.1
8,Hip Hop/Rap,20,1.9
9,Jazz,14,1.3


#### Observations:
- Here, we see the top 10 track sales by genre, in the USA.
- Of the four new artist choices, **we recommend** adding the albums from **Red Tone (Punk), Slim Jim Bites (Blues), and Meteor and the Girls (Pop)**.
- They are of genres in the top 7 for tracks sold in the USA.
- The **vast majority** of tracks sold in the USA are of the **Rock genre**, so the company should be on the lookout for new Rock records to list.

## Analyzing Employee Sales Performance

In [7]:
# Defining SQL query as string
query = '''
WITH customer_dollars AS
   (
   SELECT *
     FROM customer as c
          INNER JOIN invoice as i
          ON i.customer_id = c.customer_id
   )

SELECT e.first_name || ' '||  e.last_name as employee_name, 
       SUM(cd.total) as total_sales, 
       date(e.hire_date) as hire_date,
       CAST(JULIANDAY() - JULIANDAY(e.hire_date) AS INTEGER) AS days_employed,
       ROUND((JULIANDAY() - JULIANDAY(e.hire_date))/365, 2) AS years_employed,
       ROUND(SUM(cd.total)/(JULIANDAY() - JULIANDAY(e.hire_date))*365, 2) AS sales_per_year
       
  FROM employee as e
       INNER JOIN customer_dollars as cd
       ON cd.support_rep_id = e.employee_id
 WHERE e.title = 'Sales Support Agent'
 GROUP BY employee_name
'''
# Querying database with run_query function
run_query(query)

,employee_name,total_sales,hire_date,days_employed,years_employed,sales_per_year
0,Jane Peacock,1731.51,2017-04-01,2103,5.76,300.44
1,Margaret Park,1584.00,2017-05-03,2071,5.68,279.09
2,Steve Johnson,1393.92,2017-10-17,1904,5.22,267.13


#### Observations:
- All of the Sales Agents have been employed by Chinook for between 5 and 6 years.
- The top-selling agent's sales are ~11% higher per year than those of the bottom-selling agent.  That amount of variation seems reasonable among the small sales force.
- **Individualized sales target volume incentives are recommended**, versus targeting the top or bottom earner.

In [8]:
# Creating VIEW country aggregates with "Other" group for only 1 customer
command = '''
CREATE VIEW country_summary_other AS
    WITH country_summary AS
             (
             SELECT c.country,
                    CASE
                         WHEN COUNT(DISTINCT(c.customer_id)) = 1 THEN 'Other'
                         ELSE c.country
                     END AS country_name,
                    COUNT(DISTINCT(c.customer_id)) AS total_customers,
                    SUM(i.total) AS total_sales,
                    ROUND(AVG(total),2) AS avg_order_value
               FROM customer as c
                    INNER JOIN invoice as i
                    ON i.customer_id = c.customer_id
              GROUP BY c.country
              )
    SELECT country_name,
           SUM(total_customers) AS total_customers,
           SUM(total_sales) AS total_sales,
           ROUND(AVG(avg_order_value), 2) AS avg_order_value
      FROM country_summary
     GROUP BY country_name
     ORDER BY total_sales DESC;
'''
# Creating VIEW with run_command function
run_command(command)

# Coercing 'Other' to last row, ordered by total_sales
query = '''
SELECT country_name,
       total_customers,
       total_sales,
       avg_order_value,
       ROUND(total_sales / total_customers, 2) AS avg_sales_per_customer
  FROM (
       SELECT cso.*,
              CASE
                   WHEN country_name = 'Other' THEN 1
                   ELSE 0
               END AS sort
         FROM country_summary_other AS cso
       )
 ORDER BY sort ASC;
'''
# Running outer query with run_query function
run_query(query)

,country_name,total_customers,total_sales,avg_order_value,avg_sales_per_customer
0,USA,13,1040.49,7.94,80.04
1,Canada,8,535.59,7.05,66.95
2,Brazil,5,427.68,7.01,85.54
3,France,5,389.07,7.78,77.81
4,Germany,4,334.62,8.16,83.66
5,Czech Republic,2,273.24,9.11,136.62
6,United Kingdom,3,245.52,8.77,81.84
7,Portugal,2,185.13,6.38,92.57
8,India,2,183.15,8.72,91.57
9,Other,15,1094.94,7.44,73.00


#### Observations:
- USA has the leading sales, followed by Canada, indicating an established customer-base in these countries.
- Czech Repubic, Portugal, and India are emerging markets, with opportunity for maximizing growth to reach an established level, while continued marketing to maximize potential in Brazil, France, and Germany is also warranted.
- Further analysis of ROI from previous marketing expenditure between these two groups could help pinpoint where to invest marketing dollars.

## Albums vs Individual Tracks

In [9]:
# Creating query to compare sets of expected track_ids if invoice had full albums with actual track_ids of invoice 
# to determine if album purchase or not; categorizes invoice as album or not and returns count and percentage of 
# invoices for each category
query = '''
WITH 
     invoice_first_track AS
         (
         SELECT il.invoice_id,
                MIN(il.track_id) AS track_id
           FROM invoice_line AS il
          GROUP BY il.invoice_id
         )


SELECT album_purchased,
       COUNT(invoice_id) AS total_invoices,
       ROUND(COUNT(invoice_id)/ (
                                SELECT CAST(COUNT(ift2.invoice_id) AS FLOAT) 
                                  FROM invoice_first_track AS ift2) * 100, 1) AS percentage
  FROM

(SELECT ift.*,
       CASE
            WHEN
                 (
                 SELECT t2.track_id 
                   FROM track as t2 
                  WHERE t2.album_id = (
                                      SELECT t.album_id 
                                        FROM track as t 
                                       WHERE t.track_id = ift.track_id
                                      ) 
                 EXCEPT

                 SELECT il2.track_id
                   FROM invoice_line AS il2
                  WHERE il2.invoice_id = ift.invoice_id
                 )   IS NULL
 
                AND

                (
                SELECT il2.track_id
                  FROM invoice_line AS il2
                 WHERE il2.invoice_id = ift.invoice_id

                EXCEPT

                SELECT t2.track_id 
                  FROM track as t2 
                 WHERE t2.album_id = (
                                     SELECT t.album_id 
                                       FROM track as t 
                                      WHERE t.track_id = ift.track_id
                                     ) 
                )   IS NULL
           THEN 'yes'
           ELSE 'no'
            END AS 'album_purchased'
FROM invoice_first_track AS ift)

GROUP BY album_purchased
;
'''
# Running query with run_query function
run_query(query)

,album_purchased,total_invoices,percentage
0,no,500,81.4
1,yes,114,18.6


#### Observations:
- Over 81% of invoices are for purchases of individual tracks.
- Though in the minority, nearly 19% of purchases are for full albums.
- There is real concern for alienating these customers who prefer to purchase the full album, if that option is no longer available.  We must consider that music buffs will search for unavailable tracks elsewhere which will create churn.
- Further in-depth analysis of any potential cost savings by listing only the most popular tracks is warranted before any such move.